In [87]:
import os
import pandas as pd
import numpy as np
import librosa
from os import listdir
from os.path import isfile, join, normpath
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import f1_score, confusion_matrix, classification_report
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

In [76]:
RANDOM_STATE = 26
TEST_SIZE = 0.3
N_MFCC = 58
LS_CSV_PATH = "../data/HLS-CMDS/LS.csv"
LS_AUDIO_PATH = "../data/HLS-CMDS/LS/LS"

In [77]:
audioFiles = [f for f in listdir(LS_AUDIO_PATH) if isfile(join(LS_AUDIO_PATH, f))]
print(f"Number of audio files found: {len(audioFiles)}")
print(f"First 5 audio files: {audioFiles[:5]}")

Number of audio files found: 52
First 5 audio files: ['.DS_Store', 'F_CC_LLA.wav', 'F_CC_LMA.wav', 'F_CC_LUA.wav', 'F_CC_RLA.wav']


In [78]:
wavfiles = [f for f in audioFiles if f.endswith('.wav')]
print(f"Number of .wav files found: {len(wavfiles)}")
print(f"First 5 .wav files: {wavfiles[:5]}")

Number of .wav files found: 50
First 5 .wav files: ['F_CC_LLA.wav', 'F_CC_LMA.wav', 'F_CC_LUA.wav', 'F_CC_RLA.wav', 'F_CC_RMA.wav']


In [79]:
wavfiles = [LS_AUDIO_PATH + '/' + f for f in wavfiles]
print(f"Number of .wav files found: {len(wavfiles)}")
print(f"First 5 .wav files: {wavfiles[:5]}")

Number of .wav files found: 50
First 5 .wav files: ['../data/HLS-CMDS/LS/LS/F_CC_LLA.wav', '../data/HLS-CMDS/LS/LS/F_CC_LMA.wav', '../data/HLS-CMDS/LS/LS/F_CC_LUA.wav', '../data/HLS-CMDS/LS/LS/F_CC_RLA.wav', '../data/HLS-CMDS/LS/LS/F_CC_RMA.wav']


In [80]:
df = pd.read_csv(LS_CSV_PATH)
print(df.head())

  Gender Lung Sound Type Location Lung Sound ID
0      M          Normal      RUA       M_N_RUA
1      F          Normal      LUA       F_N_LUA
2      F          Normal      RMA       F_N_RMA
3      F          Normal      LMA       F_N_LMA
4      M          Normal      RLA       M_N_RLA


In [81]:
df['audio_path'] = df['Lung Sound ID'].apply(lambda x: normpath(join(LS_AUDIO_PATH, f"{str(x).strip()}.wav").replace('\\', '/')))

In [82]:
df['audio_path'] = df['audio_path'].str.replace('_C_', '_FC_').str.replace('_G_', '_CC_')

In [83]:
def extract_mfcc(filename, n_mfcc=N_MFCC):
    y, sr = librosa.load(filename, sr=44100)
    mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=n_mfcc)
    mfcc_mean = np.mean(mfcc, axis=1)
    mfcc_std = np.std(mfcc, axis=1)
    return np.concatenate([mfcc_mean, mfcc_std])

def flatten_mfcc(mfcc):
    return np.array(mfcc).flatten()

In [84]:
mfcc_list = []
labels = []

for _, row in df.iterrows():
    try:
        mfcc = extract_mfcc(row['audio_path'])
        # mfcc_flat = flatten_mfcc(mfcc)
        mfcc_list.append(mfcc)
        labels.append(row['Gender'])
    except Exception as e:
        print(f"[WARN] Failed: {row['audio_path']} -> {e}")
    
X = pd.DataFrame(mfcc_list)
y = pd.Series(labels)

Xtrain, Xtest, ytrain, ytest = train_test_split(X, y, test_size=TEST_SIZE, random_state=RANDOM_STATE, stratify=y)

In [85]:
pipe_model = Pipeline(steps=[
    ('scaler', StandardScaler()),
    ('svc', SVC(kernel='rbf', random_state=RANDOM_STATE, class_weight='balanced'))
])

param_grid = {
    'svc__C': [0.1, 1, 10, 100],
    'svc__gamma': [0.001, 0.01, 0.1, 1],
    'svc__kernel': ['rbf', 'linear', 'poly']
}

model_grid = GridSearchCV(
    estimator=pipe_model, 
    param_grid=param_grid, 
    scoring='f1_macro', 
    cv=5, 
    n_jobs=-1, 
    verbose=2
)

In [86]:
model_grid.fit(Xtrain, ytrain)
model = model_grid.best_estimator_
print(f"Best parameters: {model_grid.best_params_}")

ypred = model.predict(Xtest)
f1 = f1_score(ytest, ypred, average='macro')
print(f"LS: Gender -> macro-F1: {f1}")

Fitting 5 folds for each of 48 candidates, totalling 240 fits
Best parameters: {'svc__C': 0.1, 'svc__gamma': 0.1, 'svc__kernel': 'poly'}
LS: Gender -> macro-F1: 0.5982142857142857


In [88]:
cm = confusion_matrix(ytest, ypred)
print("Confusion Matrix:")
print(cm)

print("\nClassification Report:")
print(classification_report(ytest, ypred))

Confusion Matrix:
[[4 2]
 [4 5]]

Classification Report:
              precision    recall  f1-score   support

           F       0.50      0.67      0.57         6
           M       0.71      0.56      0.62         9

    accuracy                           0.60        15
   macro avg       0.61      0.61      0.60        15
weighted avg       0.63      0.60      0.60        15

